Real Time Radar Parameter Optimizer
===================================

Carson Anderson & Calvin Henggeler  
ASEN 5264 Decision Making Under Uncertainty - Spring 2025  
Semester Project  

In [14]:
using POMDPs
using POMDPTools: DiscreteUpdater, ImplicitDistribution, RolloutSimulator, HistoryRecorder, Deterministic, UnderlyingMDP, EpsGreedyPolicy, RandomPolicy, Uniform, obs_weight
using QuickPOMDPs: QuickPOMDP
using POMDPTesting: has_consistent_distributions
using QMDP: QMDPSolver
using Plots
using Statistics: std
using POMDPPolicies: alphavectors, FunctionPolicy
using Random, Distributions
using ParticleFilters
include("radarFunctions.jl")
include("radarSimulator.jl")

plot_wavefield (generic function with 1 method)

## 1. Create Smart Radar POMDP

### Environment Parameters

In [15]:
   # ===================
    # --- ENVIRONMENT ---
    # ===================

    # --- Environment Parameters ---
    global c = 3*10^8  # Speed of light
    global f = 1e9     # Frequency
    global λ = c/f     # Wavelength
    global x_max_size = 10e3     # 10 km x 10 km area
    global y_max_size = 10e3
    global radar_location = (5e3, 5e3)  # Radar location (x, y) in meters
    global rng = MersenneTwister(69420)

    # # Environment Grid
    # x_max_size = 10000.0
    # y_max_size = 10000.0    
    # divisions  = 500
    # x = collect(LinRange(-x_max_size, x_max_size, divisions))
    # y = collect(LinRange(-y_max_size, y_max_size, divisions))
    # global env = RadarEnvironment(x, y)

    # # Receiver (same as transmitter)
    # pos_rx = SVector(0.0, 0.0)
    # snapped_pos_rx = snap_to_grid(env.grid_x, pos_rx)
    # rx = Receiver(snapped_pos_rx, Float64[], 3.0)
    # add_receiver!(env, rx)

MersenneTwister(69420)

### Function POMDP

In [ ]:
# states = [(x, y) for x in 0:100:10000, y in 0:100:10000],
# flat_states = vec(states)  # flatten 2D matrix to 1D array
# n = length(flat_states)
# weights = fill(1.0 / n, n)

function_radar = QuickPOMDP(

    obstype = Tuple{Float64, Float64},    # Observation: [belief_x, belief_y]
    
    # obs_weight = 1,
    obs_weight = function (s, a, sp, o)  # Observation weight function
        # Calculate the distance between the observation and the state
        dist = sqrt((o[1] - s[1])^2 + (o[2] - s[2])^2)
        return exp(-dist^2 / (2 * 1000^2))  # Gaussian weight with std deviation of 1000
    end,
    statetype = Tuple{Float64, Float64},  # State: (x, y)

    states = [(x, y) for x in -x_max_size/2:100.0: x_max_size/2, y in y_max_size/2:100.0:y_max_size/2],

    # --- ACTION SPACE ---
    actions = [(steering_angle, beamwidth, power) for steering_angle in 1:5:355 for beamwidth in 10:10:50 for power in [1, 10, 50, 100]],

    gen = function (s, a, rng)
        steer_ang, beamwidth, tx_power = a
        xpos, ypos = s

        # model some physics
        true_range = sqrt((xpos - radar_location[1])^2 + (ypos - radar_location[2])^2)
        true_heading =  atand(xpos - radar_location[1], ypos - radar_location[2])

        # Observation Measurments
        if (steer_ang-beamwidth/2 < true_heading) && (true_heading < steer_ang+beamwidth/2)
            # Target is within beamwidth 
            delay   = radar_return_delay(true_range)
        else
            # Target is outside beamwidth, model picking up noise at random distance
            delay   = radar_return_delay(rand(range(100, stop=10000)))
        end
        obs_range   = radar_range(delay, noise_std = 0.6e-6)  # ms
    
        # Noisy estimate of position
        belief_state = boresight_polar_2_cartesian(obs_range, steer_ang + randn() * beamwidth / 2)

        # Tracking error (Euclidean)
        tracking_error = sqrt((xpos - belief_state[1])^2 + (ypos - belief_state[2])^2)
    
        if tracking_error < 50.0
            reward = 100.0
            println("Tracking success! error = ", tracking_error)
        else
            # Normalized costs
            power_cost = tx_power / 100e3
            norm_tracking_error = tracking_error / sqrt(2 * (x_max_size^2 + y_max_size^2))
            reward = -100 * (power_cost + norm_tracking_error)
        end

        return (sp=(xpos, ypos), o=belief_state, r=reward)
    end,

    # Transition for underlying MDP
    transition = function (s,a)
        return s
    end,

    # TINITIAL STATE GENERATION
    # Target starts at either far side edge of the coverage area (x ∈ [0, 100e3], y ∈ [0, 100e3])
    # target will aways start at y = 50e3, ending at a random y at the other side, gives relative heading
    # Absolute velocity is uniformy distributed from 50 - 300 m/s, v_x and v_x derived from heading and absolute velocity
    
    # function generate_initial_target(rng::AbstractRNG=Random.default_rng())
    #     # 1. Start on left or right edge
    #     start_x = rand(rng, [0.0, 100e3])  # 0 or 100,000 m
    
    #     # 2. Always start at y = 50 km
    #     start_y = 50e3  # meters
    
    #     # 3. Pick random end y on the opposite side
    #     end_y = rand(rng) * 100e3  # [0, 100_000] m
    
    #     # 4. Compute heading angle from start to end
    #     dx = (start_x == 0.0) ? 100e3 : -100e3
    #     dy = end_y - start_y
    #     heading_rad = atan(dy, dx)  # angle in radians
    
    #     # 5. Choose absolute speed [50, 300] m/s
    #     speed = rand(rng, Uniform(50.0, 300.0))
    
    #     # 6. Derive velocity components
    #     vx = speed * cos(heading_rad)
    #     vy = speed * sin(heading_rad)
    
    #     return (x=start_x, y=start_y, vx=vx, vy=vy)b
    # end
    
    # initiatial_state generate_initial_target()

    # initialstate = ImplicitDistribution(rng -> begin
    #     # init_x = 10000 * randn(rng)
    #     # init_y = 10000 * randn(rng)

    #     return (init_x, init_y) # Tuple state
    # end),

    initialstate = Uniform([(x, y) for x in 0:100.0:10000.0, y in 0:100.0:10000.0]),

    discount = 0.99,

    # isterminal = s -> false  # Tracking problem — no natural terminal state 
    # TODO: termainal states will be when the target reaches the end of the radar simulator
)


QuickPOMDP{Base.UUID("4fcbc5b2-130f-4dbd-a0ef-5818fe9b587f"), Tuple{Float64, Float64}, Tuple{Int64, Int64, Int64}, Tuple{Float64, Float64}, @NamedTuple{stateindex::Dict{Tuple{Float64, Float64}, Int64}, isterminal::Bool, states::Matrix{Tuple{Float64, Float64}}, obs_weight::var"#56#64", statetype::DataType, discount::Float64, actions::Vector{Tuple{Int64, Int64, Int64}}, gen::var"#61#69", obstype::DataType, actionindex::Dict{Tuple{Int64, Int64, Int64}, Int64}, transition::var"#62#70", initialstate::Uniform{Set{Tuple{Float64, Float64}}}}}((stateindex = Dict((800.0, 5000.0) => 59, (500.0, 5000.0) => 56, (-200.0, 5000.0) => 49, (1800.0, 5000.0) => 69, (-3700.0, 5000.0) => 14, (-2000.0, 5000.0) => 31, (-4300.0, 5000.0) => 8, (-100.0, 5000.0) => 50, (-4200.0, 5000.0) => 9, (-600.0, 5000.0) => 45…), isterminal = false, states = [(-5000.0, 5000.0); (-4900.0, 5000.0); … ; (4900.0, 5000.0); (5000.0, 5000.0);;], obs_weight = var"#56#64"(), statetype = Tuple{Float64, Float64}, discount = 0.99, actio

### Radar POMDP

In [17]:
# sim_radar = QuickPOMDP(
    
#     # ====================
#     # --- ACTION SPACE ---
#     # ====================

#     # statetype = Vector{Float64},  # State: [x, y]
#     obstype = Tuple{Float64, Float64},    # Observation: [belief_x, belief_y]
#     statetype = Tuple{Float64, Float64},  # State: (x, y)

#     discount = 0.95,

#     actions = [(steering_angle, beamwidth, power) for steering_angle in 0:5:355 for beamwidth in 5:5:45 for power in [1, 10, 50, 100]],
    

#     transition = function (s, a)
#         # For now, this is identity — modify for real motion later
#         return s
#     end,

#     observation = function (pomdp, a, s)
#         steer_ang, beamwidth, tx_power = a
#         xpos, ypos = pomdp
    
#         # Build and inject transmitter
#         tx = PointTransmitter(SVector(0.0, 0.0), 1e9, tx_power, 0.0, 0.6, steer_ang, beamwidth, false)
#         add_transmitter!(env, tx)
    
#         # Propagate environment
#         for t = 1:95
#             step!(env, 1.0)
#         end
#         rm_transmitter!(env, tx)
    
#         # Simulated radar measurements
#         powers = env.receivers[1].received_power
#         noisy_powers = powers .+ abs.(1e-11 .* randn(length(powers)))
#         return_power, return_delay = findmax(noisy_powers)
    
#         range = radar_range(return_delay * 1e-3, noise_std = 0.6e-6)  # ms → s
    
#         # Noisy estimate of position
#         belief_state = boresight_polar_2_cartesian(range, steer_ang + randn() * beamwidth / 2)
    
#         return belief_state
#     end,


#     reward = function (s, a, belief_state)
#         xpos, ypos = s
#         steer_ang, beamwidth, tx_power = a
    
#         # Tracking error (Euclidean)
#         tracking_error = sqrt((xpos - belief_state[1])^2 + (ypos - belief_state[2])^2)
    
#         if tracking_error < 100
#             return 100.0
#         else
#             # Normalized costs
#             power_cost = tx_power / 100e3
#             norm_tracking_error = tracking_error / sqrt(2 * (x_max_size^2 + y_max_size^2))
#             return -100 * (power_cost + norm_tracking_error)
#         end
#     end,


#     # ========================
#         # INITIAL STATE GENERATION
#     # ========================
    
#     # initialstate = ImplicitDistribution([10000*randn(), 10000*randn()]),

#     initialstate = ImplicitDistribution(rng -> begin
#         init_x = 10000 * randn(rng)
#         init_y = 10000 * randn(rng)
    
#         # Reflector setup (optional but allowed)
#         pos_ref = SVector(init_x, init_y)
#         snapped_pos_ref = snap_to_grid(env.grid_x, pos_ref)
#         rx_ref = Reflector(snapped_pos_ref, 0.9, 1e10, false)
#         add_reflector!(env, rx_ref)
    
#         return (init_x, init_y)  # Tuple state
#     end),

#     isterminal = s -> false  # Tracking problem — no natural terminal state 
#     # TODO: termainal states will be when the target reaches the end of the radar simulator
# )


### POMDP Testing

In [18]:
@show discount(function_radar)
@show actions(function_radar)[100]
@show rand(initialstate(function_radar))

discount(function_radar) = 0.99
(actions(function_radar))[100] = (21, 50, 100)
rand(initialstate(function_radar)) = (7300.0, 3900.0)


(7300.0, 3900.0)

In [19]:
# Test evaluation: policy alway looks forward
policy = FunctionPolicy(o->POMDPs.actions(function_radar)[1])
sim = RolloutSimulator(max_steps=2)
POMDPs.simulate(sim, function_radar, policy)

-70.3668812620111

### 2. Create Updater (Particle Filter)

The following code creates the pomdp model and the associated particle filter and runs a simulation producing a history.

In [20]:
using BasicPOMCP
using DiscreteValueIteration
using ParticleFilters

In [21]:
pomdp = function_radar
num_particles = 2000
up_radar = BootstrapFilter(pomdp, num_particles)
# up_radar = DiscreteUpdater(pomdp)

BasicParticleFilter{ParticleFilters.NormalizedESSConditionalResampler{typeof(low_variance_sample)}, ParticleFilters.POMDPPredictor{QuickPOMDP{Base.UUID("4fcbc5b2-130f-4dbd-a0ef-5818fe9b587f"), Tuple{Float64, Float64}, Tuple{Int64, Int64, Int64}, Tuple{Float64, Float64}, @NamedTuple{stateindex::Dict{Tuple{Float64, Float64}, Int64}, isterminal::Bool, states::Matrix{Tuple{Float64, Float64}}, obs_weight::var"#56#64", statetype::DataType, discount::Float64, actions::Vector{Tuple{Int64, Int64, Int64}}, gen::var"#61#69", obstype::DataType, actionindex::Dict{Tuple{Int64, Int64, Int64}, Int64}, transition::var"#62#70", initialstate::Uniform{Set{Tuple{Float64, Float64}}}}}}, ParticleFilters.POMDPReweighter{QuickPOMDP{Base.UUID("4fcbc5b2-130f-4dbd-a0ef-5818fe9b587f"), Tuple{Float64, Float64}, Tuple{Int64, Int64, Int64}, Tuple{Float64, Float64}, @NamedTuple{stateindex::Dict{Tuple{Float64, Float64}, Int64}, isterminal::Bool, states::Matrix{Tuple{Float64, Float64}}, obs_weight::var"#56#64", statetyp

In [22]:
# Custom update function for the particle filter

function POMDPs.update(up, b::ParticleCollection, a, o, model)
    # rng = MersenneTwister(69420)
    N = 2000  # number of particles
    new_particles = Vector{typeof(first(b))}()
    weights = Float64[]

    while length(new_particles) < N
        s = rand(rng, b)  # sample a particle
        gen_result = @gen(:sp, :o)(model, s, a, rng)  # (sp, o, r)
        sp = gen_result.sp      # state does not move
        o_sim = gen_result.o    # simulated obervation (belief state)

        # Weight particle inversly to euclidean distance to actual
        obs_err = 1 / sqrt((sp[1] - o_sim[1])^2 + (sp[2] - o_sim[2])^2)  # Euclidean distance
        
        w = exp(-obs_err^2 / (2 * 100^2))  # 100 m std dev (tune as needed)

        if w > 1e-10
            push!(new_particles, sp)
            push!(weights, w)
        end
    end

    # plot particles belief locations
    plot(first.(new_particles), last.(new_particles), label="belief", xlabel="x", ylabel="y", lw=2, marker=:circle)


    # Resample new particle set using low variance resampling
    belief = WeightedParticleBelief(new_particles, weights, sum(weights), nothing)
    return ParticleFilters.resample(LowVarianceResampler(N), belief, rng)
end

# struct UnweightedPF{M<:POMDP} <: POMDPs.Updater
#     m::M
#     n_particles::Int
# end


# function POMDPs.update(up::UnweightedPF, b::ParticleCollection, a, o, model)
#     rng = MersenneTwister(69420)
#     N = 5000  # number of particles
#     new_particles = Vector{typeof(first(b))}()
#     weights = Float64[]


#     # Create new particles by simulating
#     while length(new_particles) < N
#         s = rand(rng, b)  # sample a particle
#         gen_result = @gen(:sp, :o)(model, s, a, rng)  # (sp, o, r)
#         sp = gen_result.sp      # state does not move
#         o_sim = gen_result.o    # simulated obervation (belief state)

#         # Weight particle inversly to euclidean distance to actual
#         obs_err = 1 / sqrt((sp[1] - o_sim[1])^2 + (sp[2] - o_sim[2])^2)  # Euclidean distance
        
#         w = exp(-obs_err^2 / (2 * 100^2))  # 100 m std dev (tune as needed)

#         if w > 1e-10
#             push!(new_particles, sp)
#             push!(weights, w)
#         end
#     end

#     # plot particles belief locations
#     plot(first.(new_particles), last.(new_particles), label="belief", xlabel="x", ylabel="y", lw=2, marker=:circle)


#     # Resample new particle set using low variance resampling
#     belief = WeightedParticleBelief(new_particles, weights, sum(weights), nothing)
#     return ParticleFilters.resample(LowVarianceResampler(N), belief, rng)
# end

### 3. Create A Policy

In [27]:
struct HeuristicPolicy{M<:POMDP} <: POMDPs.Policy
    epsilon::Float64
end

function POMDPs.action(p::HeuristicPolicy, b) # b is any distribution, i.e. anything that has pdf defined

    # Estimate the state
    n = length(points)
    avg_x = sum(p[1] for p in b) / n
    avg_y = sum(p[2] for p in b) / n
    s_hat = (avg_x, avg_y)
    
    #action that would give best rewards for estimated state
    power = 1
    steering = radar_view_angle(s_hat[1],s_hat[2], radar_location)
    BW = 10

    return (steering, BW, power)
end 

# rollout_policy = HeuristicPolicy(pomdp)

function pomcp_solve(m) # this function makes capturing m in the rollout policy more efficient
    solver = POMCPSolver(tree_queries=5000000,
                         c=1,
                         default_action=RandomPolicy(m),
                         estimate_value=FORollout(FunctionPolicy(s-> action(HeuristicPolicy(pomdp), s))),
                         max_time=10,
                         )
    return solve(solver, m)
end
policy = pomcp_solve(pomdp)
# policy = FunctionPolicy(o->rand(actions(pomdp)))
# policy = EpsGreedyPolicy(pomdp, 0.2)


POMCPPlanner{QuickPOMDP{Base.UUID("4fcbc5b2-130f-4dbd-a0ef-5818fe9b587f"), Tuple{Float64, Float64}, Tuple{Int64, Int64, Int64}, Tuple{Float64, Float64}, @NamedTuple{stateindex::Dict{Tuple{Float64, Float64}, Int64}, isterminal::Bool, states::Matrix{Tuple{Float64, Float64}}, obs_weight::var"#56#64", statetype::DataType, discount::Float64, actions::Vector{Tuple{Int64, Int64, Int64}}, gen::var"#61#69", obstype::DataType, actionindex::Dict{Tuple{Int64, Int64, Int64}, Int64}, transition::var"#62#70", initialstate::Uniform{Set{Tuple{Float64, Float64}}}}}, BasicPOMCP.SolvedFORollout{FunctionPolicy{var"#85#86"}, TaskLocalRNG}, BasicPOMCP.var"#3#7", TaskLocalRNG}(POMCPSolver
  max_depth: Int64 20
  c: Float64 1.0
  tree_queries: Int64 5000000
  max_time: Float64 10.0
  tree_in_info: Bool false
  default_action: RandomPolicy{TaskLocalRNG, QuickPOMDP{Base.UUID("4fcbc5b2-130f-4dbd-a0ef-5818fe9b587f"), Tuple{Float64, Float64}, Tuple{Int64, Int64, Int64}, Tuple{Float64, Float64}, @NamedTuple{stateind

### 4. Create Simulator

In [28]:
rs = RolloutSimulator(rng=rng, max_steps=300)
hs = HistoryRecorder(max_steps=100)


HistoryRecorder(TaskLocalRNG(), false, false, 100, nothing)

### Run Simulation

In [29]:
simulate(hs, pomdp, policy, up_radar)

Tracking success! error = 249.000000340391
Tracking success! error = 157.9999994588127
Tracking success! error = 452.32576489208225
Tracking success! error = 128.65584062364368
Tracking success! error = 374.65251893293186
Tracking success! error = 473.2353098931452
Tracking success! error = 262.63385182115
Tracking success! error = 340.68390050615307
Tracking success! error = 381.3012666429837
Tracking success! error = 489.59921130681363
Tracking success! error = 219.92064397011274
Tracking success! error = 320.7696354632779
Tracking success! error = 474.6599203686593
Tracking success! error = 378.52422380212187
Tracking success! error = 382.8827105932041
Tracking success! error = 27.363807545073215
Tracking success! error = 100.68686540510818
Tracking success! error = 339.75011714520394
Tracking success! error = 93.46889918398705
Tracking success! error = 292.38499120999506
Tracking success! error = 271.7587620877108
Tracking success! error = 403.5940507683846
Tracking success! error 

Excessive output truncated after 524297 bytes.

322.69862338042077
Tracking success! error = 293.78075391189395
Tracking success! error = 259.81263710511547
Tracking success! error = 331.29692154470507
Tracking success! error = 388.94504191324603
Tracking success! error = 430.76251118648923
Tracking success! error = 339.0950973155341
Tracking success! error = 418.42195009352866
Tracking success! error = 484.34660392914867
Tracking success! error = 295.1388400595662
Tracking success! error = 343.38360286567666
Tracking success! error = 163.01024581610878
Tracking success! error = 258.490932870506
Tracking success! error = 392.86646684232164
Tracking success! error = 392.4383165674543
Tracking success! error = 466.8281058616627
Tracking success! error = 330.5967896294249
Tracking success! error = 416.9903262059347
Tracking success! error = 271.3208044281118
Tracking success! error = 488.8295945542156
Tracking success! error = 373.3046981816225
Tracking success! error = 274.74405330633004
Tracking success! error = 84.55222852118507
Tra

100-element POMDPTools.Simulators.SimHistory{@NamedTuple{s::Tuple{Float64, Float64}, a::Tuple{Int64, Int64, Int64}, sp::Tuple{Float64, Float64}, o::Tuple{Float64, Float64}, r::Float64, info::Nothing, t::Int64, action_info::Dict{Symbol, Any}, b::WeightedParticleBelief{Tuple{Float64, Float64}}, bp::WeightedParticleBelief{Tuple{Float64, Float64}}, update_info::Nothing}, Float64}:
 (s = (3700.0, 5600.0), a = (316, 30, 10), sp = (3700.0, 5600.0), o = (-893.7597704569749, 1343.9454869157007), r = -31.32159084389284, info = nothing, t = 1, action_info = Dict(:exception => MethodError(HeuristicPolicy, (QuickPOMDP{Base.UUID("4fcbc5b2-130f-4dbd-a0ef-5818fe9b587f"), Tuple{Float64, Float64}, Tuple{Int64, Int64, Int64}, Tuple{Float64, Float64}, @NamedTuple{stateindex::Dict{Tuple{Float64, Float64}, Int64}, isterminal::Bool, states::Matrix{Tuple{Float64, Float64}}, obs_weight::var"#56#64", statetype::DataType, discount::Float64, actions::Vector{Tuple{Int64, Int64, Int64}}, gen::var"#61#69", obstype::

In [26]:
@doc HistoryRecorder

A simulator that records the history for later examination

The simulation will be terminated when either

1. a terminal state is reached (as determined by `isterminal()` or
2. the discount factor is as small as `eps` or
3. max_steps have been executed

Keyword Arguments:     - `rng`: The random number generator for the simulation     - `capture_exception::Bool`: whether to capture an exception and store it in the history, or let it go uncaught, potentially killing the script     - `show_progress::Bool`: show a progress bar for the simulation     - `eps`     - `max_steps`

Usage (optional arguments in brackets):

```
hr = HistoryRecorder()
history = simulate(hr, pomdp, policy, [updater [, init_belief [, init_state]]])
```
